In [ ]:
# Import members.txt scraped file

In [1]:
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
import calendar
import itertools

from datetime import date, datetime, timedelta

In [3]:
with open ("members_2019_11_23.txt", "r") as myfile:
    data=myfile.read()

In [4]:
x = data.split('\n3-dots-h\n')
y = [w.replace('\n', '; ') for w in x]
y = y[1:]
a = [w.replace('; Message', '') for w in y]
df1 =pd.DataFrame(a)
df2 = df1[0].str.split(';',expand=True)
df2.rename(columns = {0: "NAME", 1:"OTHER1", 2: "OTHER2", 3:"OTHER3", 4:"OTHER4"}, inplace = True)

In [319]:
pd.DataFrame(df2).to_csv('members_raw_'+ date.today().strftime('%Y_%m_%d')+'.csv')

In [5]:
b = [w.replace('New Member; ', '') for w in a]
b = [w.replace('Admin; ', '') for w in b]
b = [re.sub(r'View.+posts?; ', '', w) for w in b]
df3 =pd.DataFrame(b)
df3 = df3[0].str.split(';',expand=True)
df3.rename(columns = {0: "NAME", 1:"JOINED", 2: "OTHER"}, inplace = True)

In [317]:
pd.DataFrame(df3).to_csv('members_clean_'+ date.today().strftime('%Y_%m_%d')+'.csv')

### Isolating Dates

In [6]:
#Added by:
df3[['JOINED1', 'JOINED2']] = df3["JOINED"].str.split(' on',expand=True)
df3['NAME']= df3["NAME"].str.strip()
df3['JOINED']= df3["JOINED"].str.strip()
df3['OTHER']= df3["OTHER"].str.strip()
df3['JOINED1']= df3["JOINED1"].str.strip()
df3['JOINED2']= df3["JOINED2"].str.strip()


In [7]:
df3["DATES1"] = pd.to_datetime(df3["JOINED2"],errors='coerce')

In [8]:
#dates with hours
df3.loc[df3['JOINED'].str.contains('hour'), 'h'] = 0

#dates with weekdays
xtoday = [0,1,2,3,4,5,6]

for j in xtoday:    
    if date.today().weekday() > j: 
        df3.loc[df3['JOINED'].str.contains(calendar.day_name[j]), 'wd'] = date.today().weekday() - j
    elif date.today().weekday() < j: 
        df3.loc[df3['JOINED'].str.contains(calendar.day_name[j]), 'wd'] = date.today().weekday() - (j-7)                
    else:        
        df3.loc[df3['JOINED'].str.contains(calendar.day_name[j]), 'wd'] = 7

#dates with interval = 1
df3.loc[df3['JOINED'].str.contains('Yesterday'), 'yes'] = 1
df3.loc[df3['JOINED'] == 'Joined about a week ago', 'w1'] = 7
df3.loc[df3['JOINED'] == 'Joined about a month ago', 'm1'] = 30
df3.loc[df3['JOINED'] == 'Joined about a year ago', 'y1'] = 365

#dates with interval >1
df3['d'] = df3[df3['JOINED'].str.contains("days")]["JOINED"].str.extract("(\d*\.?\d+)", expand=False)
df3['w'] = df3[df3['JOINED'].str.contains("weeks")]["JOINED"].str.extract("(\d*\.?\d+)", expand=False)
df3['m'] = df3[df3['JOINED'].str.contains("months")]["JOINED"].str.extract("(\d*\.?\d+)", expand=False)
df3['y'] = df3[df3['JOINED'].str.contains("years")]["JOINED"].str.extract("(\d*\.?\d+)", expand=False)
df3['d'] = df3['d'].astype(float) * 1
df3['w'] = df3['w'].astype(float) * 7
df3['m'] = df3['m'].astype(float) * 30
df3['y'] = df3['y'].astype(float) * 365


In [9]:
df3['numbers']=df3['h'].combine_first(df3['wd'])
df3['numbers']=df3['numbers'].combine_first(df3['yes'])
df3['numbers']=df3['numbers'].combine_first(df3['w1'])
df3['numbers']=df3['numbers'].combine_first(df3['m1'])
df3['numbers']=df3['numbers'].combine_first(df3['y1'])
df3['numbers']=df3['numbers'].combine_first(df3['d'])
df3['numbers']=df3['numbers'].combine_first(df3['w'])
df3['numbers']=df3['numbers'].combine_first(df3['m'])
df3['numbers']=df3['numbers'].combine_first(df3['y'])

In [10]:
dfint = DataFrame(df3['numbers'].dropna())
dfint['DATES2']=[date.today() - timedelta(i) for i in dfint['numbers']]


In [11]:
df3 ['DATESALL'] = df3['DATES1'].combine_first(dfint['DATES2'])

In [12]:
df= df3[['NAME', 'DATESALL','JOINED','OTHER']]

In [322]:
pd.DataFrame(df).to_csv('members_join_dates_'+ date.today().strftime('%Y_%m_%d')+'.csv')